In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#  In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort?
# Casper

 

In [2]:
df_movies = pd.read_csv('../../data/movie.csv')
df_movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


## 2. Data processing 
Alleen 'keywords' en 'genres' nodig.

In [3]:

# Ongewenste kolommen verwijderen
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'color': 'Colour',
                          'director_name': 'Director',
                          'num_critic_for_reviews': 'Number of critics',
                          'duration': 'Duration',
                          'director_facebook_likes': 'Director FB likes',
                          'actor_3_facebook_likes': 'Actor 3 FB likes',
                          'actor_2_name': 'Actor 2 name',
                          'actor_1_facebook_likes': 'Actor 1 FB likes',
                          'gross': 'Gross',
                          'genres': 'Genres',
                          'actor_1_name': 'Actor 1 name',
                          'movie_title': 'Movie title',
                          'num_voted_users': 'Number of voted users',
                          'cast_total_facebook_likes': 'Total Cast FB likes',
                          'actor_3_name': 'Actor 3 name',
                          'facenumber_in_poster': 'Number of faces on poster',
                          'plot_keywords': 'Plot Keywords',
                          'num_user_for_reviews': 'Number of user reviews',
                          'language': 'Language',
                          'country': 'Country',
                          'content_rating': 'Age rating',
                          'budget': 'Budget',
                          'title_year': 'Release year',
                          'actor_2_facebook_likes': 'Actor 2 FB likes',
                          'imdb_score': 'IMDB Score',
                          'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen
df_movies = df_movies[['Movie title',
                       'Release year',
                       'Director',
                       'Director FB likes',
                       'Gross',
                       'Budget',
                       'Duration',
                       'Language',
                       'Country',
                       'Colour',
                       'Genres',
                       'IMDB Score',
                       'Number of voted users',
                       'Number of critics',
                       'Number of user reviews',
                       'Age rating',
                       'Total Cast FB likes',
                       'Movie FB likes',
                       'Actor 1 name',
                       'Actor 2 name',
                       'Actor 3 name',
                       'Actor 1 FB likes',
                       'Actor 2 FB likes',
                       'Actor 3 FB likes',
                       'Plot Keywords',
                       'Number of faces on poster',
                       ]]

# Datatypen aanpassen
# 1. Floats omzetten naar integers
#  De dataset bevat geen kolommen die dienen te worden bewaard als float, behalve `IMDB Score`
df_movies_IMDB_Score = df_movies["IMDB Score"]  # Tijdelijke kopie van de kolom `IMDB Score`
df_movies = df_movies.drop('IMDB Score', axis=1).fillna(0).astype(int, errors='ignore') # Waarden omzetten naar integers
df_movies.insert(11, "IMDB Score", df_movies_IMDB_Score)  # `IMDB Score` weer toevoegen aan originele DataFrame
del df_movies_IMDB_Score

# 2. De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')


## 3. Data Cleaning

In [4]:

# NaN-types verwijderen
df_movies.dropna(inplace=True)

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard

# Negatieve waardes verwijderen
num = df_movies._get_numeric_data()
num[num < 0] = 0


## Q2. Genres voorspellen gebaseerd op Plot Keywords
De onderzoeksvraag gaat als volgt: 
> In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort? 
### Q2. Data collection
Bij `Q2` wordt er gebruik gemaakt van de kolommen `Genres` en `Plot Keywords` uit `df_movies`. Om ervoor te zorgen dat bij andere onderzoeksvragen de DataFrame niet onnodig wordt aangepast, wordt de dataframe voor de zekerheid als een kopie aangeroepen. 

In [5]:
df_Q2 = df_movies.loc[:, ("Genres", "Plot Keywords")].copy()
df_Q2.head()

,Genres,Plot Keywords
4810,Drama|History|War,huguenot|intolerance|medicis|protestant|wedding
4958,Crime|Drama,family relationships|gang|idler|poorhouse|thief
4885,Drama|Romance|War,chewing gum|climbing a tree|france|translation...
2734,Drama|Sci-Fi,art deco|bible quote|dance|silent film|worker
4812,Musical|Romance,sibling rivalry|singer|sister act|whistling|wi...


### Q2. Data processing
Aangezien vrijwel alle machine learning algoritmen alleen algebraïsche datatypes accepteren, moeten zowel `Plot Keywords` als `Genres` ingrijpend veranderd worden. Beide zijn namelijk categoriale datatypes.


#### Q2. Cleaning voor processing


Na een korte inspectie van de data, kom je een aantal waardes van 0 en “0” tegen (`int` en `string`). Dit zijn missende waarden. 

Hoewel dit gewoonlijks in stap 3 van het Data Science proces gebeurt, is het in deze scenario beter om dit van tevoren te doen. De 0 en "0" zijn na deze stap namelijk moeilijker terug te vinden.



In [6]:
# All '0' are missing values, remove
df_Q2.replace("0", np.NaN, inplace=True)
df_Q2.replace(0, np.NaN, inplace=True)
df_Q2.dropna(inplace=True)

#### Q2. Terug naar processing
Om in de gebruikte modellen gebruik te maken van `Genres` en `Plot Keywords` moeten deze op twee manieren worden gesplitst. Hoewel `Split Keywords` niet wordt gebruikt als input variabele, is deze alsnog handig om te hebben bij het analyseren van de data. 

In [7]:
# Split "Genres" and "Keywords" by "|", creating list types
df_Q2.loc[:, "Split Genres"] = df_Q2["Genres"].str.split(pat="|")
df_Q2.loc[:, "Split Keywords"] = df_Q2["Plot Keywords"].str.split(pat="|")

# Replace "|" with " " to create strings analysable by TF-IDf.
df_Q2.loc[:, "Plot Keywords"] = df_Q2["Plot Keywords"].str.replace("|", " ")

df_Q2.head()

,Genres,Plot Keywords,Split Genres,Split Keywords
4810,Drama|History|War,huguenot intolerance medicis protestant wedding,"[Drama, History, War]","[huguenot, intolerance, medicis, protestant, w..."
4958,Crime|Drama,family relationships gang idler poorhouse thief,"[Crime, Drama]","[family relationships, gang, idler, poorhouse,..."
4885,Drama|Romance|War,chewing gum climbing a tree france translation...,"[Drama, Romance, War]","[chewing gum, climbing a tree, france, transla..."
2734,Drama|Sci-Fi,art deco bible quote dance silent film worker,"[Drama, Sci-Fi]","[art deco, bible quote, dance, silent film, wo..."
4812,Musical|Romance,sibling rivalry singer sister act whistling wi...,"[Musical, Romance]","[sibling rivalry, singer, sister act, whistlin..."


### Q2. Data cleaning
Al het Data Cleaning is voor deze stap al gedaan, namelijk in de paragrafen `3. Data Cleaning` en `Q2. Cleaning voor processing`.

## 4. Data Exploration & Analysis

Ten eerste is het van belang om te kijken naar het aantal Genres en Plot Keywords. Plot Keywords worden hier nog gezien als hele zinnen.

In [8]:
pd.DataFrame([df_Q2["Split Genres"].str.len().describe(), df_Q2["Split Keywords"].str.len().describe()])

,count,mean,std,min,25%,50%,75%,max
Split Genres,4673.0,2.898138,1.191393,1.0,2.0,3.0,4.0,8.0
Split Keywords,4673.0,4.930452,0.460884,1.0,5.0,5.0,5.0,5.0


Er zijn gemiddeld drie genres en vijf plot keywords per film. Het is echter géén uniforme data. Zowel genres als plot keywords heeft een minimum van 1 woord (I.E. er is maar één genre of één plot keyword). Het maximaal aantal waarden voor genres is 8, het maximaal aantal waarden voor plot keywords is daarentegen 5. 

De volgende stap is om te gaan kijken naar hoeveel unieke genres en plot keywords er zijn. 


In [9]:
def find_uniques(se, dtype = "category"):
    name = se.name
    flatten = lambda l: [item for sublist in l for item in sublist]
    
    unique_values = se.values
    
    unique_values = np.unique(flatten(unique_values), return_counts=True)
    
    
    d = {name : unique_values[0], "Count" : unique_values[1]}
    types = {name : dtype, "Count" : "int32"}
    
    df_unique = pd.DataFrame(d).astype(types)
    
    return df_unique.sort_values("Count", ascending=False)

genre_uniques = find_uniques(df_Q2["Split Genres"])
print(genre_uniques.size)
genre_uniques

48


,Split Genres,Count
7,Drama,2396
4,Comedy,1779
21,Thriller,1309
0,Action,1078
17,Romance,1051
1,Adventure,869
5,Crime,823
18,Sci-Fi,572
9,Fantasy,563
12,Horror,518


Het gaat hierbij dus om 48 unieke genres. 

Zowel genres als plot keywords is categorische data. Aangezien meeste machine learning modellen algebraïsch zijn, accepteren deze alleen numerieke data. Dat betekent dat zowel genres als keywords omgezet moet worden naar numerieke data. Dit wordt meestal doormiddel van ‘One-Hot’ encoding gedaan. Dit is echter een probleem wanneer er sprake is van een grote variatie aan data. 

Bij one-hot encoding, er van uitgaand dat we ieder plot keyword gebruiken, houdt dat 7975 extra kolommen in. Voor plot keywords is dit echter nog tot in zekere mate op te lossen met behulp van term frequency-inverse document frequency (TF-IDF). 

TF-IDF genereerd waardes en kolommen gebaseerd op de frequentie van woorden in een string, ook kan TF-IDF veel gebruikte, maar weinig zeggende woorden (stop words) als ‘of’ en ‘and’ uit een tekst halen.


In [10]:
key_uniques = find_uniques(df_Q2["Split Keywords"])
# for i in range(50):
#     print(i, key_uniques[key_uniques["Count"] > i].size)
# key_uniques[key_uniques["Count"] > i]
key_uniques.describe()

,Count
count,7975.000000
mean,2.889028
std,6.606527
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,189.000000


Een tweede probleem is dat genres multilabel waardes zijn, een film heeft meer dan een genre. Alleen het eerste genre pakken is een mogelijkheid, maar dit zal de accuratesse van ons model zeer negatief beïnvloeden. De genres zijn namelijk niet gesorteerd op toepasbaarheid, maar alfabetisch. 

In [11]:
alphabetical = True
for value in df_Q2["Split Genres"].values:
    if value != sorted(value):
        alphabetical = False
        
alphabetical
    

True

Films zoals “[Bad Boys](https://www.imdb.com/title/tt0112442/?ref_=ttkw_kw_tt)” met genres als ‘Action, Comedy, Crime’ zullen dan alleen geclassificeerd worden als ‘Action’. [Plot keywords](https://www.imdb.com/title/tt0112442/keywords?ref_=tt_ql_stry_4) zoals ‘evil man’ en ‘firearm’ komen dan nog goed overeen met het genre, maar plot keywords als ‘buddy movie’ en ‘buddy comedy’ niet.

Om dit op te lossen, moet er gebruik gemaakt worden van ‘multi-label classification’. Dit is niet te verwarren met ‘multi-class classification’. Bij multi-label is er sprake van dat meerdere labels tegelijkertijd toepasbaar kunnen zijn, bij multi-class is er altijd maar één klasse(label) toepasbaar.


### Omzetten van lijsten naar one-hot

In [12]:
def one_hot(df_main, column):
    df = df_main.copy()
    uniques = find_uniques(df[column])
    
    for cat in uniques[column].values:
        df[cat] = df[column].apply(lambda x : cat in x).astype("int32")
        
    return df
      
        
df_Q2 = one_hot(df_Q2, "Split Genres")
df_Q2.head()

,Genres,Plot Keywords,Split Genres,Split Keywords,Drama,Comedy,Thriller,Action,Romance,Adventure,...,Music,War,History,Sport,Musical,Documentary,Western,Film-Noir,Short,News
4810,Drama|History|War,huguenot intolerance medicis protestant wedding,"[Drama, History, War]","[huguenot, intolerance, medicis, protestant, w...",1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4958,Crime|Drama,family relationships gang idler poorhouse thief,"[Crime, Drama]","[family relationships, gang, idler, poorhouse,...",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4885,Drama|Romance|War,chewing gum climbing a tree france translation...,"[Drama, Romance, War]","[chewing gum, climbing a tree, france, transla...",1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2734,Drama|Sci-Fi,art deco bible quote dance silent film worker,"[Drama, Sci-Fi]","[art deco, bible quote, dance, silent film, wo...",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4812,Musical|Romance,sibling rivalry singer sister act whistling wi...,"[Musical, Romance]","[sibling rivalry, singer, sister act, whistlin...",0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

vec = TfidfVectorizer(stop_words=stop_words)
# vec = TfidfVectorizer()
X = vec.fit_transform(df_Q2["Plot Keywords"])





In [14]:

pd.DataFrame(X.toarray(), columns=vec.get_feature_names()).head()

,007,10,1000000,11,1190s,12,12th,13,130,13th,...,zero,zeus,zodiac,zoloft,zombie,zone,zoo,zookeeper,zoologist,zorro
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model building


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_Q2[["Plot Keywords"] + genre_uniques["Split Genres"].tolist()], random_state=2)

train, validation = train_test_split(train, random_state=2)

X_train = train["Plot Keywords"]
Y_train = train[genre_uniques["Split Genres"].tolist()]

X_test = test["Plot Keywords"]
Y_test = test[genre_uniques["Split Genres"].tolist()]

X_validation = validation["Plot Keywords"]

print(train.shape)
print(test.shape)
print(validation.shape)

(2628, 25)
(1169, 25)
(876, 25)


In [16]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
dic = {}

for genre in genre_uniques["Split Genres"]:
    NB_pipeline.fit(X_train, train[genre])
    
    
    dic[genre] = NB_pipeline.predict(X_test)
    

pd.DataFrame(dic, index=X_test.index).sample(10)

,Drama,Comedy,Thriller,Action,Romance,Adventure,Crime,Sci-Fi,Fantasy,Horror,...,Music,War,History,Sport,Musical,Documentary,Western,Film-Noir,Short,News
4655,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1446,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3877,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3103,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3487,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2906,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1733,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# df_Q2.loc[[4567]]
df_predict = pd.DataFrame(dic, index=X_test.index)
df_temp = (test[genre_uniques["Split Genres"].tolist()] == df_predict)
df_temp.describe().T["freq"] / df_temp.describe().T["count"] 

Drama          0.681779
Comedy         0.697177
Thriller       0.749358
Action         0.788708
Romance        0.788708
Adventure      0.825492
Crime          0.829769
Sci-Fi         0.873396
Fantasy        0.875107
Horror         0.893071
Family         0.896493
Mystery        0.906758
Biography      0.948674
Animation      0.959795
Music          0.964927
War            0.952096
History        0.959795
Sport          0.962361
Musical        0.972626
Documentary    0.975192
Western        0.983747
Film-Noir             1
Short          0.998289
News                  1
dtype: object

In [18]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier

# classifier = BinaryRelevance(GaussianNB())
classifier = BinaryRelevance(MultinomialNB())
print(X_train.values.shape, Y_train.shape)

vec = TfidfVectorizer(stop_words=stop_words)
vec.fit(X_train, X_test)
X_train = vec.transform(X_train)

# train
classifier.fit(X_train, Y_train)

(2628,) (2628, 24)


BinaryRelevance(classifier=MultinomialNB(alpha=1.0, class_prior=None,
                                         fit_prior=True),
                require_dense=[True, True])

In [19]:
predictions = classifier.predict(vec.transform(X_test))

In [20]:
from sklearn.metrics import precision_score
accuracy_dict = []
for i, genre in enumerate(genre_uniques["Split Genres"]):
    accuracy_dict.append({"Genre" : genre, "Accuracy Score" : accuracy_score(Y_test[genre], predictions[:, i].A), "Precision Score" : precision_score(Y_test[genre], predictions[:, i].A)})
#     print(f"Genre: {genre}; Accuracy score: {accuracy_score(Y_test[genre], predictions[:, i].A)}; Precision score: {precision_score(Y_test[genre], predictions[:, i].A)}")

pd.DataFrame(accuracy_dict)



C:\Users\Caspe\Anaconda3\envs\movie-revenue-predictions\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,Genre,Accuracy Score,Precision Score
0,Drama,0.681779,0.642857
1,Comedy,0.697177,0.751174
2,Thriller,0.749358,0.768116
3,Action,0.788708,0.862745
4,Romance,0.788708,0.785714
5,Adventure,0.825492,0.892857
6,Crime,0.829769,1.000000
7,Sci-Fi,0.873396,1.000000
8,Fantasy,0.875107,1.000000
9,Horror,0.893071,1.000000


In [21]:
from sklearn.metrics import hamming_loss
hamming_loss(Y_test, predictions)

0.10486170516110636